# Abordagem dos 4 subsistemas
## Imports e configurações

In [29]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler

# Carrega o dataframe a partir de um arquivo CSV
df_processed = pd.read_csv("./itu-classificado.csv", low_memory=False)

# Garante ordenação temporal
df = df_processed.sort_values("timestamp").reset_index(drop=True)

# Algumas das features atualmente não são utilizadas, mas estão aqui para fins de testes com subsistemas futuros
FEATURES = ["Eng_RPM","Cool_T","Oil_P","Oil_L","Recalque","Succao","Bat_V","Char_V"]

# seq params
SEQ  = 60
STRIDE = 5
TEST_RATIO = 0.2

def make_seq(X, y, seq=SEQ, stride=STRIDE):
    Xs, ys = [], []
    for i in range(0, len(X)-seq, stride):
        Xs.append(X[i:i+seq])
        ys.append(y[i+seq])
    return np.array(Xs, dtype=np.float32), np.array(ys, dtype=np.float32)



## Indicadores de resgate por subsistema

In [30]:
def clip01(x): 
    return np.clip(x, 0.0, 1.0)
def minmax01(x, lo, hi):
    return clip01((x - lo) / (hi - lo + 1e-9))

COOL_T_MAX = 92.0
BAT_V_MIN, BAT_V_MAX = 24.0, 28.0
CHAR_V_MIN, CHAR_V_MAX = 25.0, 29.0

oil_p = df["Oil_P"].to_numpy(float)
oil_l = df["Oil_L"].to_numpy(float)
rec   = df["Recalque"].to_numpy(float)
suc   = df["Succao"].to_numpy(float)

# Lubrificação
oil_p01 = minmax01(oil_p, np.nanquantile(oil_p,0.01), np.nanquantile(oil_p,0.99))
oil_l01 = minmax01(oil_l, np.nanquantile(oil_l,0.01), np.nanquantile(oil_l,0.99))
target_lube = 100.0 * (0.6*oil_p01 + 0.4*oil_l01)

# Hidráulico
rec01 = minmax01(rec, np.nanquantile(rec,0.01), np.nanquantile(rec,0.99))
suc01 = minmax01(-np.abs(suc), np.nanquantile(-np.abs(suc),0.01), np.nanquantile(-np.abs(suc),0.99))
target_hyd = 100.0 * clip01(0.7*rec01 + 0.3*suc01)

# Matriz final de targets (2 saídas)
Y_multi = np.stack([target_lube, target_hyd], axis=1).astype(np.float32)


### Split, scaler e sequências

In [31]:
X_all = df[FEATURES].to_numpy(dtype=np.float32)

cut = int(len(X_all) * (1 - TEST_RATIO))
X_tr_raw, X_te_raw = X_all[:cut], X_all[cut:]
Y_tr_raw, Y_te_raw = Y_multi[:cut], Y_multi[cut:]

scaler_subs = MinMaxScaler()
X_tr = scaler_subs.fit_transform(X_tr_raw)
X_te = scaler_subs.transform(X_te_raw)

X_train_m, Y_train_m = make_seq(X_tr, Y_tr_raw)
X_test_m,  Y_test_m  = make_seq(X_te, Y_te_raw)

X_train_m.shape, X_test_m.shape, Y_train_m.shape

((80936, 60, 8), (20225, 60, 8), (80936, 2))

### Gru multi-target

In [32]:
import tensorflow as tf
from keras import Sequential
from keras.layers import GRU, Dense, Dropout

tf.keras.utils.set_random_seed(42)

model_subs = Sequential([
    GRU(64, return_sequences=True, input_shape=(X_train_m.shape[1], X_train_m.shape[2])),
    Dropout(0.2),
    GRU(64),
    Dropout(0.2),
    Dense(2, activation="linear")
])
model_subs.compile(optimizer="adam", loss="mae", metrics=["mae"])

hist_subs = model_subs.fit(
    X_train_m, Y_train_m,
    validation_data=(X_test_m, Y_test_m),
    epochs=15, batch_size=64, verbose=1
)

Y_pred = np.clip(model_subs.predict(X_test_m), 0.0, 100.0)

Epoch 1/15


/home/ec07-g3/Desktop/repo_projeto/2025-2A-T12-EC07-G03/.venv/lib/python3.12/site-packages/keras/src/layers/rnn/rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


1265/1265 ━━━━━━━━━━━━━━━━━━━━ 12s 9ms/step - loss: 10.7383 - mae: 10.7383 - val_loss: 7.6423 - val_mae: 7.6423
Epoch 2/15
1265/1265 ━━━━━━━━━━━━━━━━━━━━ 10s 8ms/step - loss: 6.2045 - mae: 6.2045 - val_loss: 4.2513 - val_mae: 4.2513
Epoch 3/15
1265/1265 ━━━━━━━━━━━━━━━━━━━━ 11s 9ms/step - loss: 3.5660 - mae: 3.5660 - val_loss: 2.0509 - val_mae: 2.0509
Epoch 4/15
1265/1265 ━━━━━━━━━━━━━━━━━━━━ 11s 9ms/step - loss: 2.5406 - mae: 2.5406 - val_loss: 1.5476 - val_mae: 1.5476
Epoch 5/15
1265/1265 ━━━━━━━━━━━━━━━━━━━━ 11s 9ms/step - loss: 2.2754 - mae: 2.2754 - val_loss: 1.1784 - val_mae: 1.1784
Epoch 6/15
1265/1265 ━━━━━━━━━━━━━━━━━━━━ 11s 9ms/step - loss: 2.1568 - mae: 2.1568 - val_loss: 1.4978 - val_mae: 1.4978
Epoch 7/15
1265/1265 ━━━━━━━━━━━━━━━━━━━━ 10s 8ms/step - loss: 2.0811 - mae: 2.0811 - val_loss: 1.1410 - val_mae: 1.1410
Epoch 8/15
1265/1265 ━━━━━━━━━━━━━━━━━━━━ 10s 8ms/step - loss: 2.0460 - mae: 2.0460 - val_loss: 0.8068 - val_mae: 0.8068
Epoch 9/15
1265/1265 ━━━━━━━━━━━━━━━━━━━━

### Métricas por subsistema

In [33]:
from sklearn.metrics import mean_absolute_error
try:
    from sklearn.metrics import root_mean_squared_error
    rmse_fn = root_mean_squared_error
except:
    from sklearn.metrics import MeanSquaredError
    rmse_fn = lambda a,b: MeanSquaredError(squared=False)(a,b)

names = ["Lubrificação","Hidráulico"]
for i, name in enumerate(names):
    mae  = mean_absolute_error(Y_test_m[:,i], Y_pred[:,i])
    rmse = rmse_fn(Y_test_m[:,i], Y_pred[:,i])
    print(f"[{name}] MAE={mae:.2f} RMSE={rmse:.2f}")

[Lubrificação] MAE=0.72 RMSE=2.38
[Hidráulico] MAE=0.70 RMSE=1.84


### Salvar modelo

In [34]:
import joblib
model_subs.save("nn_subsistemas_4heads.h5")
joblib.dump(scaler_subs, "scaler_subs.pkl")
print("Salvos: nn_subsistemas_4heads.h5, scaler_subs.pkl")

Salvos: nn_subsistemas_4heads.h5, scaler_subs.pkl


## Tuning de hiperparâmetros

In [37]:
import keras_tuner as kt
from keras import Sequential
from keras.layers import GRU, Dense, Dropout

def build_gru_model(hp):
    model = Sequential()
    model.add(
        GRU(
            units=hp.Int("units", min_value=32, max_value=128, step=32),
            return_sequences=True,
            input_shape=(X_train_m.shape[1], X_train_m.shape[2])
        )
    )
    model.add(Dropout(hp.Choice("dropout1", [0.2, 0.3, 0.5])))
    model.add(
        GRU(
            units=hp.Int("units2", min_value=32, max_value=128, step=32)
        )
    )
    model.add(Dropout(hp.Choice("dropout2", [0.2, 0.3, 0.5])))
    model.add(Dense(2, activation="linear"))

    model.compile(
        optimizer=keras.optimizers.Adam(
            learning_rate=hp.Choice("lr", [1e-2, 1e-3, 1e-4])
        ),
        loss="mae",
        metrics=["mae"]
    )
    return model


In [39]:
tuner = kt.RandomSearch(
    build_gru_model,
    objective="val_mae",  # queremos minimizar o MAE
    max_trials=10,        # número de combinações de hiperparâmetros a testar
    executions_per_trial=1,
    overwrite=True,
    directory="tuner_logs",
    project_name="subsistemas_gru"
)

tuner.search_space_summary()


Search space summary
Default search space size: 5
units (Int)
{'default': None, 'conditions': [], 'min_value': 32, 'max_value': 128, 'step': 32, 'sampling': 'linear'}
dropout1 (Choice)
{'default': 0.2, 'conditions': [], 'values': [0.2, 0.3, 0.5], 'ordered': True}
units2 (Int)
{'default': None, 'conditions': [], 'min_value': 32, 'max_value': 128, 'step': 32, 'sampling': 'linear'}
dropout2 (Choice)
{'default': 0.2, 'conditions': [], 'values': [0.2, 0.3, 0.5], 'ordered': True}
lr (Choice)
{'default': 0.01, 'conditions': [], 'values': [0.01, 0.001, 0.0001], 'ordered': True}


/home/ec07-g3/Desktop/repo_projeto/2025-2A-T12-EC07-G03/.venv/lib/python3.12/site-packages/keras/src/layers/rnn/rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [40]:
tuner.search(
    X_train_m, Y_train_m,
    validation_data=(X_test_m, Y_test_m),
    epochs=10, batch_size=64
)

best_hps = tuner.get_best_hyperparameters(1)[0]
print("Melhores hiperparâmetros encontrados:")
print("units1:", best_hps.get("units"))
print("units2:", best_hps.get("units2"))
print("dropout1:", best_hps.get("dropout1"))
print("dropout2:", best_hps.get("dropout2"))
print("lr:", best_hps.get("lr"))


Trial 10 Complete [00h 01m 52s]
val_mae: 2.073190450668335

Best val_mae So Far: 0.6834219694137573
Total elapsed time: 00h 18m 47s
Melhores hiperparâmetros encontrados:
units1: 96
units2: 128
dropout1: 0.2
dropout2: 0.3
lr: 0.001


In [41]:
best_model = tuner.hypermodel.build(best_hps)

history = best_model.fit(
    X_train_m, Y_train_m,
    validation_data=(X_test_m, Y_test_m),
    epochs=20, batch_size=64
)

Y_pred = np.clip(best_model.predict(X_test_m), 0.0, 100.0)

Epoch 1/20


1265/1265 ━━━━━━━━━━━━━━━━━━━━ 11s 8ms/step - loss: 9.4720 - mae: 9.4720 - val_loss: 6.5007 - val_mae: 6.5007
Epoch 2/20
1265/1265 ━━━━━━━━━━━━━━━━━━━━ 11s 9ms/step - loss: 4.5862 - mae: 4.5862 - val_loss: 2.2367 - val_mae: 2.2367
Epoch 3/20
1265/1265 ━━━━━━━━━━━━━━━━━━━━ 11s 9ms/step - loss: 2.5480 - mae: 2.5480 - val_loss: 1.3347 - val_mae: 1.3347
Epoch 4/20
1265/1265 ━━━━━━━━━━━━━━━━━━━━ 12s 9ms/step - loss: 2.1610 - mae: 2.1610 - val_loss: 0.9979 - val_mae: 0.9979
Epoch 5/20
1265/1265 ━━━━━━━━━━━━━━━━━━━━ 11s 9ms/step - loss: 2.0486 - mae: 2.0486 - val_loss: 0.9583 - val_mae: 0.9583
Epoch 6/20
1265/1265 ━━━━━━━━━━━━━━━━━━━━ 12s 9ms/step - loss: 1.9893 - mae: 1.9893 - val_loss: 0.8258 - val_mae: 0.8258
Epoch 7/20
1265/1265 ━━━━━━━━━━━━━━━━━━━━ 11s 9ms/step - loss: 1.9530 - mae: 1.9530 - val_loss: 0.7719 - val_mae: 0.7719
Epoch 8/20
1265/1265 ━━━━━━━━━━━━━━━━━━━━ 11s 9ms/step - loss: 1.9020 - mae: 1.9020 - val_loss: 1.1244 - val_mae: 1.1244
Epoch 9/20
1265/1265 ━━━━━━━━━━━━━━━━━━━━ 1

In [47]:
from sklearn.metrics import mean_absolute_error, root_mean_squared_error, r2_score

names = ["Lubrificação", "Hidráulico"]

for i, name in enumerate(names):
    mae  = mean_absolute_error(Y_test_m[:, i], Y_pred[:, i])
    rmse = root_mean_squared_error(Y_test_m[:, i], Y_pred[:, i])
    r2   = r2_score(Y_test_m[:, i], Y_pred[:, i])
    print(f"[{name}] MAE={mae:.2f} RMSE={rmse:.2f} R²={r2:.3f}")

[Lubrificação] MAE=0.66 RMSE=2.14 R²=0.981
[Hidráulico] MAE=0.53 RMSE=1.45 R²=0.991
